## URL Grabber

This assignment develops a simple url grabber. The program gets as input two address files, one containing urls and the other one contains regular expressions. The purpose of the program is to check the urls, fetch their context and match with given regular expressions.

- docopt is used to get as arguments two file names: url_file and regex_file
- pandas is used to read urls assuming urls have big data
- multi-processing is used

In [1]:
from solution import multi_process_urls

from function_utils import is_match, fetch_text_from_url

In [2]:
multi_process_urls("url_file.txt", "regex_file.txt", 5)

.... started processing the urls .....
processing url:  https://en.wikipedia.org/wiki/Python_(programming_language)
processing url:  https://en.wikipedia.org/wiki/CMA-ES
{'worker_id': 1, 'time': 'Wed Nov 27 17:58:38 2019', 'RE': 'Hello(.*)', 'error': 0, 'matched-string': 'something', 'deltatime': 0.0, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 0, 'time': 'Wed Nov 27 17:58:38 2019', 'RE': 'welcome(.*)', 'error': 0.9999555750805819, 'matched-string': '', 'deltatime': 0.313221, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 2, 'time': 'Wed Nov 27 17:58:38 2019', 'RE': 'This is Me(.*)', 'error': 0.9999407672126778, 'matched-string': '', 'deltatime': 0.438308, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 3, 'time': 'Wed Nov 27 17:58:38 2019', 'RE': 'derivatives nor even', 'error': 0.9999062161421763, 'matched-string': '', 'deltatim

In [3]:
multi_process_urls("url_file.txt", "regex_file.txt", 1)

.... started processing the urls .....
processing url:  https://en.wikipedia.org/wiki/Python_(programming_language)
{'worker_id': 0, 'time': 'Wed Nov 27 17:59:12 2019', 'RE': 'welcome(.*)', 'error': 0.9999555750805819, 'matched-string': '', 'deltatime': 0.309218, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 0, 'time': 'Wed Nov 27 17:59:12 2019', 'RE': 'Hello(.*)', 'error': 0, 'matched-string': 'something', 'deltatime': 0.0, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 0, 'time': 'Wed Nov 27 17:59:13 2019', 'RE': 'This is Me(.*)', 'error': 0.9999407672126778, 'matched-string': '', 'deltatime': 0.407294, 'url': 'https://en.wikipedia.org/wiki/Python_(programming_language)', 'url-code': 0}
{'worker_id': 0, 'time': 'Wed Nov 27 17:59:13 2019', 'RE': 'derivatives nor even', 'error': 0.9999062161421763, 'matched-string': '', 'deltatime': 0.349258, 'url': 'https://en.wikipedia.org/wiki/Py